## Pytorch

In [ ]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        간단한 RNN 모델 구현
        Args:
            input_size: 입력 특성의 크기
            hidden_size: 은닉 상태의 크기
            output_size: 출력의 크기
        """
        super(SimpleRNN, self).__init__()

        self.hidden_size = hidden_size

        # RNN 층
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True
        )

        # 출력 층
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden=None):
        """
        순전파
        Args:
            x: 입력 텐서, shape: (batch_size, sequence_length, input_size)
            hidden: 초기 은닉 상태 (선택사항)
        """
        # 초기 은닉 상태가 없으면 0으로 초기화
        if hidden is None:
            hidden = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)

        # RNN 층을 통과
        output, hidden = self.rnn(x, hidden)

        # 모든 시점의 출력값을 전결합층에 통과
        output = self.fc(output)

        return output, hidden

# 모델 사용 예시
def example_usage():
    # 하이퍼파라미터 설정
    batch_size = 32
    sequence_length = 10
    input_size = 5
    hidden_size = 64
    output_size = 3

    # 모델 인스턴스 생성
    model = SimpleRNN(input_size, hidden_size, output_size)

    # 샘플 입력 데이터 생성
    x = torch.randn(batch_size, sequence_length, input_size)

    # 모델 실행
    output, hidden = model(x)

    # 출력 크기 확인
    print(f"출력 텐서 크기: {output.shape}")  # [batch_size, sequence_length, output_size]
    print(f"은닉 상태 크기: {hidden.shape}")  # [1, batch_size, hidden_size]

if __name__ == "__main__":
    example_usage()

출력 텐서 크기: torch.Size([32, 10, 3])
은닉 상태 크기: torch.Size([1, 32, 64])


## Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.models import Model
import numpy as np

def create_rnn_model(input_shape, hidden_units, output_units):
    """
    기본적인 RNN 모델 생성

    Args:
        input_shape: 입력 형태 (sequence_length, input_dim)
        hidden_units: RNN 은닉층의 유닛 수
        output_units: 출력층의 유닛 수

    Returns:
        컴파일된 Keras 모델
    """
    # 입력층 정의
    inputs = Input(shape=input_shape)

    # RNN 층 추가
    rnn_out = SimpleRNN(
        units=hidden_units,
        activation='relu',
        return_sequences=True
    )(inputs)

    # 출력층 추가
    outputs = Dense(units=output_units, activation='linear')(rnn_out)

    # 모델 생성
    model = Model(inputs=inputs, outputs=outputs)

    # 모델 컴파일
    model.compile(
        optimizer='nadam',
        loss='mse',
        metrics=['mae']
    )

    return model

# 학습을 위한 샘플 데이터 생성 함수
def generate_sample_data(num_samples, sequence_length, input_dim, output_dim):
    """
    테스트를 위한 샘플 시계열 데이터 생성
    """
    X = np.random.randn(num_samples, sequence_length, input_dim)
    y = np.random.randn(num_samples, sequence_length, output_dim)
    return X, y

def main():
    # 모델 파라미터 설정
    sequence_length = 10  # 시퀀스 길이
    input_dim = 5        # 입력 차원
    hidden_units = 64    # RNN 은닉층 유닛 수
    output_dim = 3       # 출력 차원

    # 모델 생성
    model = create_rnn_model(
        input_shape=(sequence_length, input_dim),
        hidden_units=hidden_units,
        output_units=output_dim
    )

    # 모델 구조 출력
    model.summary()

    # 샘플 데이터 생성
    num_samples = 1000
    X_train, y_train = generate_sample_data(
        num_samples=num_samples,
        sequence_length=sequence_length,
        input_dim=input_dim,
        output_dim=output_dim
    )

    # 모델 학습
    history = model.fit(
        X_train,
        y_train,
        epochs=10,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )

    return model, history

if __name__ == "__main__":
    model, history = main()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 10, 5)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 10, 64)              │           4,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10, 3)               │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,675 (18.26 KB)

 Trainable params: 4,675 (18.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 1.1328 - mae: 0.8432 - val_loss: 1.0556 - val_mae: 0.8211
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0314 - mae: 0.8076 - val_loss: 1.0373 - val_mae: 0.8135
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.0022 - mae: 0.7959 - val_loss: 1.0323 - val_mae: 0.8116
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.0138 - mae: 0.7991 - val_loss: 1.0291 - val_mae: 0.8099
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9840 - mae: 0.7878 - val_loss: 1.0274 - val_mae: 0.8093
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.0031 - mae: 0.7978 - val_loss: 1.0258 - val_mae: 0.8086
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9923 - mae: 0.7917 - val_loss: 1.0244 - val_mae: 0.8080
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9857 - mae: 0.7912 - val_loss: 1.0244 - val_mae: 0.8081
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9923 - mae: